In [9]:
import os
import zipfile
import tensorflow as tf

# Download the dataset from Kaggle and save in Google Colab Files
!kaggle datasets download -d prasunroy/natural-images

# Unzip the natural-images.zip
with zipfile.ZipFile('natural-images.zip', 'r') as zip_ref:
  zip_ref.extractall('')

# Define path for original dataset
data_dir = 'natural_images'

Dataset URL: https://www.kaggle.com/datasets/prasunroy/natural-images
License(s): CC-BY-NC-SA-4.0
natural-images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
import shutil
import random

# Define path for split data
split_dir = 'data_split'

# Define path for train, validation and test directory
train_dir = os.path.join(split_dir, 'train')
val_dir = os.path.join(split_dir, 'val')
test_dir = os.path.join(split_dir, 'test')

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split ratio
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Split data
categories = os.listdir(data_dir)
seed = 42

for category in categories:
  category_path = os.path.join(data_dir, category)

  if not os.path.isdir(category_path): continue

  # Create subdirectories for each class in train and val files
  os.makedirs(os.path.join(train_dir, category), exist_ok=True)
  os.makedirs(os.path.join(val_dir, category), exist_ok=True)
  os.makedirs(os.path.join(test_dir, category), exist_ok=True)

  # Get all images in the category to split
  images = os.listdir(category_path)
  random.seed(seed)
  random.shuffle(images)

  # Find indices of image to split
  total_images = len(images)
  train_images_index = int(train_ratio * total_images)
  val_images_index = int((train_ratio + val_ratio) * total_images)

  # Split data by indices
  train_images = images[:train_images_index]
  val_images = images[train_images_index:val_images_index]
  test_images = images[val_images_index:]

  # Move images to train and val directories with folders as original file:
  for image in train_images:
    shutil.copy(os.path.join(category_path, image), os.path.join(train_dir, category))

  for image in val_images:
    shutil.copy(os.path.join(category_path, image), os.path.join(val_dir, category))

  for image in test_images:
    shutil.copy(os.path.join(category_path, image), os.path.join(test_dir, category))

In [11]:
from tensorflow import keras

img_height, img_width = 227, 227
batch_size = 128

# Create Datasets
train_generator = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=seed
)

val_generator = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=seed
)

test_generator = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False
)

class_names = train_generator.class_names
num_classes = len(class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_generator = train_generator.prefetch(buffer_size=AUTOTUNE)
val_generator = val_generator.prefetch(buffer_size=AUTOTUNE)
test_generator = test_generator.prefetch(buffer_size=AUTOTUNE)

Found 4826 files belonging to 8 classes.
Found 1034 files belonging to 8 classes.
Found 1039 files belonging to 8 classes.


In [12]:
# Define VGG-like Architecture
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout

model = Sequential([
    Input(shape=(img_height, img_width, 3)),
    Conv2D(filters=64, kernel_size=(7,7), strides=4, activation="relu"),
    Conv2D(filters=64, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Flatten(),
    Dense(units=256, activation="relu"),
    Dense(units=256, activation="relu"),

    Dense(units=num_classes, activation='linear')
])

In [13]:
model.compile(
    optimizer= tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 56, 56, 64)          │           9,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 56, 56, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 56, 56, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 28, 28, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 28, 28, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 28, 28, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 256)                 │       3,211,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 8)                   │           2,056 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,400,136 (12.97 MB)

 Trainable params: 3,399,880 (12.97 MB)

 Non-trainable params: 256 (1.00 KB)

In [14]:
# Train the Model
epochs = 20
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
)

Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 14s 218ms/step - accuracy: 0.5223 - loss: 2.8165 - val_accuracy: 0.2437 - val_loss: 17.1391
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 16s 192ms/step - accuracy: 0.8724 - loss: 0.3347 - val_accuracy: 0.2901 - val_loss: 10.1423
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 180ms/step - accuracy: 0.9507 - loss: 0.1294 - val_accuracy: 0.5484 - val_loss: 2.2371
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 168ms/step - accuracy: 0.9897 - loss: 0.0406 - val_accuracy: 0.7892 - val_loss: 0.6944
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 156ms/step - accuracy: 0.9938 - loss: 0.0246 - val_accuracy: 0.8559 - val_loss: 0.4308
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 197ms/step - accuracy: 0.9979 - loss: 0.0086 - val_accuracy: 0.8810 - val_loss: 0.3857
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 147ms/step - accuracy: 0.9987 - loss: 0.0043 - val_accuracy: 0.8298 - val_loss: 0.5521
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 194ms/step - accuracy: 0.9999 - loss: 0.0036 - val_accurac

In [15]:
# Evaluate on Test Set
import numpy as np
from sklearn.metrics import classification_report

loss, accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {accuracy:.5f}")

# Classification Report
y_true = []
y_pred = []

for images, labels in test_generator:
    preds = model.predict(images, verbose=0)
    preds = np.argmax(preds, axis=1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.numpy().tolist())

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 180ms/step - accuracy: 0.8910 - loss: 0.4169
Test accuracy: 0.89798
Classification Report:
              precision    recall  f1-score   support

    airplane       0.95      0.95      0.95       110
         car       0.94      0.95      0.94       146
         cat       0.72      0.77      0.74       133
         dog       0.68      0.63      0.65       106
      flower       0.92      0.91      0.92       127
       fruit       0.99      1.00      1.00       150
   motorbike       0.96      0.97      0.96       119
      person       0.97      0.95      0.96       148

    accuracy                           0.90      1039
   macro avg       0.89      0.89      0.89      1039
weighted avg       0.90      0.90      0.90      1039

